# Running the spectral graph model using the classes branch. 

Trying out functionality to debug. 

### Import modules

In [1]:
import sys
sys.path.append("..")

#SCFC modules
from forward import network_transfer as nt
from utils import functions
from brain import Brain
from read import data_reader as dr
from preprocess import permute as perm 
from utils import path as pth

#generic modules
import matplotlib.pyplot as mpl
import numpy as np
from scipy.signal import lfilter, firls, decimate
import time

### Create an instance of a brain and do basic operations.

In [2]:
mybrain = Brain.Brain()

In [3]:
directory = pth.get_sibling_path('data')

In [4]:
mybrain.add_connectome(directory)

In [5]:
mybrain.connectome.size

7396

In [6]:
mybrain.reorder_connectome(mybrain.connectome, mybrain.distance_matrix)

In [12]:
mybrain.distance_matrix.shape

(86, 86)

In [9]:
vars(mybrain).keys()

dict_keys(['connectome', 'reducedConnectome', 'distance_matrix', 'permutation', 'ordering', 'ntf_params'])

Get the reduced form of the connectome using these mysterious functions. 

In [10]:
mybrain.bi_symmetric_c()
mybrain.reduce_extreme_dir()

### Running the network transfer model on the brain class

Do the calculation of the NTF for a single frequency.

In [15]:
fq, ev, Vv, freqresp_out, _ = nt.network_transfer_function(mybrain, 
                                                           parameters = mybrain.ntf_params, 
                                                           w=2*np.pi)

OK, as before let's do it for a range of frequencies.

In [17]:
fmin = 2 # 2Hz - 45Hz signal range
fmax = 45
fvec = np.linspace(fmin,fmax,40)

evec = []
Vvec =[]
fqall = []
freq_model = []

start = time.time()
for i in fvec:
    w = 2*np.pi*i
    fq, ev, Vv, freqresp_out, _ = nt.network_transfer_function(mybrain, mybrain.ntf_params, w)
    fqall.append(fq)
    evec.append(ev)
    Vvec.append(Vv)
    freq_model.append(freqresp_out)
    
fqall=np.asarray(fqall)
evec = np.asarray(evec)
Vvec = np.asarray(Vvec)
freq_model = np.asarray(freq_model)
freq_model = np.transpose(freq_model)
stop = time.time()
duration = stop - start

print('Computation time = ', duration)
# np.moveaxis(Vvec,1,0).shape #86x40x57 just like matlab

Computation time =  0.45694589614868164
